
# Rheology Prediction (G′ / G″) — Colab Baseline (v2: renamed columns + quality weighting)
**Changes in v2**  
- Column names updated:  
  - `Temperature_C` → **`Temp_C`**  
  - `Frequency_rad_s` → **`Freq_rad_s`**  
  - `Gprime_Pa` → **`G1_Pa`**  
  - `Gdoubleprime_Pa` → **`G2_Pa`**  
- After computing `tan_delta = G2_Pa / G1_Pa`, rows with **tan_delta > 1000** are flagged as suspicious and assigned **10× lower sample weight** for training.



## 0) Environment & Data Upload
Choose one:
1) Upload local CSV (`files.upload`)  
2) Mount Google Drive


In [ ]:

# Option A: Upload local CSV from your computer
from google.colab import files
import io, pandas as pd

uploaded = files.upload()  # Choose your CSV file (e.g., PB_Data.csv)
csv_name = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[csv_name]))
print("Loaded:", csv_name, "shape:", df.shape)
df.head()


In [ ]:

# Option B: Mount Google Drive and read from a path
# from google.colab import drive
# drive.mount('/content/drive')
# import pandas as pd
# csv_path = "/content/drive/MyDrive/your_folder/PB_Data.csv"
# df = pd.read_csv(csv_path)
# print("Loaded:", csv_path, "shape:", df.shape)
# df.head()



## 1) Basic Schema Checks (v2)
Expected columns (case-sensitive):
- `Sample_ID` (string)
- `Length_nm`, `Width_nm` (numeric)
- `Temp_C` (numeric)
- `Freq_rad_s` (numeric, >0)
- `G1_Pa`, `G2_Pa` (numeric, >0)
- `tan_delta` (optional; if absent, computed as `G2_Pa / G1_Pa`)
Also, rows with `tan_delta > 1000` will be down-weighted by 10× during training.


In [ ]:

import numpy as np

required_cols = ['Sample_ID','Length_nm','Width_nm','Temp_C','Freq_rad_s','G1_Pa','G2_Pa']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Coerce numeric types
for col in ['Length_nm','Width_nm','Temp_C','Freq_rad_s','G1_Pa','G2_Pa']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with critical NaNs
before = len(df)
df = df.dropna(subset=['Sample_ID','Length_nm','Width_nm','Temp_C','Freq_rad_s','G1_Pa','G2_Pa'])
after = len(df)
print(f"Dropped {before-after} rows due to NaNs in critical columns. Remaining:", after)

# Basic positivity checks for logs
assert (df['G1_Pa'] > 0).all(), "G1_Pa must be > 0 for log transforms"
assert (df['G2_Pa'] > 0).all(), "G2_Pa must be > 0 for log transforms"
assert (df['Freq_rad_s'] > 0).all(), "Freq_rad_s must be > 0 for log transforms"

# Compute tan_delta if not provided
if 'tan_delta' not in df.columns:
    df['tan_delta'] = df['G2_Pa'] / df['G1_Pa']

# Quality flag & sample weights
df['flag_suspicious'] = df['tan_delta'] > 1000
df['sample_weight'] = np.where(df['flag_suspicious'], 0.1, 1.0)

print("Suspicious rows (tan_delta > 1000):", int(df['flag_suspicious'].sum()))
df.head()



## 2) Feature Engineering
- `Aspect = Length_nm / Width_nm`
- `log10_freq = log10(Freq_rad_s)`
- Targets in log space: `log10_Gp = log10(G1_Pa)`, `log10_Gpp = log10(G2_Pa)`
- Interaction: `T_x_logf = Temp_C * log10_freq`


In [ ]:

df = df.copy()
df['Aspect'] = df['Length_nm'] / df['Width_nm']
df['log10_freq'] = np.log10(df['Freq_rad_s'])
df['log10_Gp'] = np.log10(df['G1_Pa'])
df['log10_Gpp'] = np.log10(df['G2_Pa'])
df['T_x_logf'] = df['Temp_C'] * df['log10_freq']

feature_cols = ['Length_nm','Width_nm','Aspect','Temp_C','log10_freq','T_x_logf']
target_cols  = ['log10_Gp','log10_Gpp']

X = df[feature_cols].values
y = df[target_cols].values
groups = df['Sample_ID'].values
weights = df['sample_weight'].values

X.shape, y.shape, weights.shape



## 3) Group-Aware Cross-Validation with Sample Weights
We group by `Sample_ID` to evaluate generalization to unseen materials.  
Suspicious rows (tan_delta > 1000) receive 10× lower weight during training.


In [ ]:

from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

gkf = GroupKFold(n_splits=5)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', MultiOutputRegressor(RandomForestRegressor(
        n_estimators=400,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    )))
])

cv_metrics = []
fold = 0
for train_idx, valid_idx in gkf.split(X, y, groups):
    fold += 1
    X_tr, X_va = X[train_idx], X[valid_idx]
    y_tr, y_va = y[train_idx], y[valid_idx]
    w_tr = weights[train_idx]

    pipe.fit(X_tr, y_tr, model__sample_weight=w_tr)
    y_pred = pipe.predict(X_va)

    # Evaluate in log space
    mae_log = mean_absolute_error(y_va, y_pred, multioutput='raw_values')
    rmse_log = np.sqrt(mean_squared_error(y_va, y_pred, multioutput='raw_values'))
    r2_log = [r2_score(y_va[:,0], y_pred[:,0]), r2_score(y_va[:,1], y_pred[:,1])]

    # Linear space
    y_va_lin = 10**y_va
    y_pred_lin = 10**y_pred
    mae_lin = mean_absolute_error(y_va_lin, y_pred_lin, multioutput='raw_values')
    rmse_lin = np.sqrt(mean_squared_error(y_va_lin, y_pred_lin, multioutput='raw_values'))

    cv_metrics.append({
        'fold': fold,
        'MAE_log10_Gp': mae_log[0],
        'MAE_log10_Gpp': mae_log[1],
        'RMSE_log10_Gp': rmse_log[0],
        'RMSE_log10_Gpp': rmse_log[1],
        'R2_log10_Gp': r2_log[0],
        'R2_log10_Gpp': r2_log[1],
        'MAE_Gp(Pa)': mae_lin[0],
        'MAE_Gpp(Pa)': mae_lin[1],
        'RMSE_Gp(Pa)': rmse_lin[0],
        'RMSE_Gpp(Pa)': rmse_lin[1],
    })

cv_df = pd.DataFrame(cv_metrics)
cv_df



## 4) Residual Diagnostics


In [ ]:

import matplotlib.pyplot as plt

pipe.fit(X, y, model__sample_weight=weights)
y_hat = pipe.predict(X)

# Residuals for log10(G′)
res_gp = y[:,0] - y_hat[:,0]
plt.figure()
plt.scatter(y_hat[:,0], res_gp, s=8)
plt.axhline(0)
plt.xlabel("Predicted log10(G′)")
plt.ylabel("Residual (true - pred)")
plt.title("Residuals for log10(G′)")
plt.show()

# Residuals for log10(G″)
res_gpp = y[:,1] - y_hat[:,1]
plt.figure()
plt.scatter(y_hat[:,1], res_gpp, s=8)
plt.axhline(0)
plt.xlabel("Predicted log10(G″)")
plt.ylabel("Residual (true - pred)")
plt.title("Residuals for log10(G″)")
plt.show()



## 5) Train Final Model & Export Artifacts
Saves: `model.pkl` (pipeline), `feature_info.json`.


In [ ]:

import joblib, json

pipe.fit(X, y, model__sample_weight=weights)
joblib.dump(pipe, "model.pkl")
feature_info = {
    "feature_cols": ['Length_nm','Width_nm','Aspect','Temp_C','log10_freq','T_x_logf'],
    "target_cols": ['log10_Gp','log10_Gpp'],
    "log_targets": True,
    "note": "Inputs expect Sample_ID, Length_nm, Width_nm, Temp_C, Freq_rad_s. Internally builds Aspect/log10_freq/T_x_logf. Suspicious rows (tan_delta>1000) down-weighted by 10x."
}
with open("feature_info.json","w") as f:
    json.dump(feature_info, f, indent=2)
print("Saved: model.pkl, feature_info.json")



## 6) Predict on New Designs
Prepare a CSV with columns: `Sample_ID, Length_nm, Width_nm, Temp_C, Freq_rad_s`.


In [ ]:

# Example prediction input template (v2)
import pandas as pd
pred_in = pd.DataFrame({
    "Sample_ID": ["N1","N1","N2"],
    "Length_nm": [120,120,150],
    "Width_nm": [30,30,35],
    "Temp_C": [25,40,25],
    "Freq_rad_s": [1.0, 10.0, 0.1],
})
pred_in.to_csv("prediction_input_example_v2.csv", index=False)
pred_in.head()


In [ ]:

# Load artifacts and predict (v2)
import joblib, numpy as np, pandas as pd

pipe = joblib.load("model.pkl")

inp = pd.read_csv("prediction_input_example_v2.csv")
inp['Aspect'] = inp['Length_nm'] / inp['Width_nm']
inp['log10_freq'] = np.log10(inp['Freq_rad_s'])
inp['T_x_logf'] = inp['Temp_C'] * inp['log10_freq']

X_new = inp[['Length_nm','Width_nm','Aspect','Temp_C','log10_freq','T_x_logf']].values
y_pred_log = pipe.predict(X_new)
y_pred = 10**y_pred_log

out = inp.copy()
out['G1_Pa_pred'] = y_pred[:,0]
out['G2_Pa_pred'] = y_pred[:,1]
out['tan_delta_pred'] = out['G2_Pa_pred'] / out['G1_Pa_pred']
out.to_csv("prediction_output_v2.csv", index=False)
out



## 7) (Optional) Hyperparameter Search (with weights)


In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GroupKFold

base_rf = RandomForestRegressor(random_state=42, n_jobs=-1)
search = RandomizedSearchCV(
    estimator=MultiOutputRegressor(base_rf),
    param_distributions={
        "estimator__n_estimators": randint(200, 800),
        "estimator__max_depth": randint(3, 20),
        "estimator__min_samples_split": randint(2, 20),
        "estimator__min_samples_leaf": randint(1, 10),
    },
    n_iter=20,
    cv=GroupKFold(n_splits=5).split(X, y, groups),
    scoring="neg_mean_absolute_error",
    random_state=42,
    n_jobs=-1,
    verbose=1
)

search.fit(X, y, estimator__sample_weight=weights)
print("Best params:", search.best_params_)



## 8) (Optional) Uncertainty Estimate via Tree Std (with weights)


In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

rf_gp = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
rf_gpp = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)

rf_gp.fit(X_scaled, y[:,0], sample_weight=weights)
rf_gpp.fit(X_scaled, y[:,1], sample_weight=weights)

def pred_with_std(model, Xs):
    preds = np.vstack([t.predict(Xs) for t in model.estimators_])
    return preds.mean(axis=0), preds.std(axis=0)

# Example using prediction_input_example_v2.csv
inp = pd.read_csv("prediction_input_example_v2.csv")
inp['Aspect'] = inp['Length_nm'] / inp['Width_nm']
inp['log10_freq'] = np.log10(inp['Freq_rad_s'])
inp['T_x_logf'] = inp['Temp_C'] * inp['log10_freq']
Xs = scaler.transform(inp[['Length_nm','Width_nm','Aspect','Temp_C','log10_freq','T_x_logf']].values)

m_gp, s_gp = pred_with_std(rf_gp, Xs)
m_gpp, s_gpp = pred_with_std(rf_gpp, Xs)

unc = inp.copy()
unc['log10_Gp_mean'] = m_gp
unc['log10_Gp_std'] = s_gp
unc['log10_Gpp_mean'] = m_gpp
unc['log10_Gpp_std'] = s_gpp

unc['Gp_Pa_mean'] = 10**unc['log10_Gp_mean']
unc['Gp_Pa_low']  = 10**(unc['log10_Gp_mean'] - 2*s_gp)
unc['Gp_Pa_high'] = 10**(unc['log10_Gp_mean'] + 2*s_gp)

unc['Gpp_Pa_mean'] = 10**unc['log10_Gpp_mean']
unc['Gpp_Pa_low']  = 10**(unc['log10_Gpp_mean'] - 2*s_gpp)
unc['Gpp_Pa_high'] = 10**(unc['log10_Gpp_mean'] + 2*s_gpp)

unc
